In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/good (226).jpg
/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/good (432).jpg
/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/good (490).jpg
/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/good (422).jpg
/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/good (361).jpg
/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/good (220).jpg
/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/good (406).jpg
/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/good (681).jpg
/kaggle/input/tyre-quality-classification/Digital images of defective and good c

In [1]:
import torch

In [2]:
import torchvision
import torchvision.models as models

In [3]:
from torchvision import transforms
from torch.utils.data import random_split, DataLoader

In [4]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])

])

In [5]:
dataset = torchvision.datasets.ImageFolder(root = r'/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres', transform=preprocess)

print(dataset.class_to_idx)
print(dataset[0])

{'defective': 0, 'good': 1}
(tensor([[[ 0.0056, -0.0116, -0.0629,  ..., -0.0458, -0.0801, -0.2171],
         [-0.0287, -0.0629, -0.0458,  ..., -0.1143, -0.1657, -0.3369],
         [-0.0287, -0.0629, -0.0801,  ..., -0.1657, -0.1828, -0.3198],
         ...,
         [-0.6965, -1.0048, -0.9877,  ..., -0.5938, -0.6281, -0.6109],
         [-0.7993, -1.0562, -0.9020,  ..., -0.5938, -0.6794, -0.6281],
         [-0.8164, -1.0733, -0.7650,  ..., -0.6623, -0.6109, -0.5938]],

        [[ 0.0826,  0.0651,  0.0126,  ..., -0.1099, -0.1275, -0.2675],
         [ 0.0476,  0.0301,  0.0476,  ..., -0.1800, -0.2150, -0.3901],
         [ 0.0651,  0.0476,  0.0301,  ..., -0.2150, -0.2325, -0.3725],
         ...,
         [-0.7227, -1.0378, -1.0028,  ..., -0.7227, -0.7577, -0.7752],
         [-0.8277, -1.0903, -0.9328,  ..., -0.7227, -0.8102, -0.7927],
         [-0.8452, -1.1078, -0.8102,  ..., -0.7752, -0.7577, -0.7577]],

        [[ 0.2173,  0.1651,  0.1128,  ...,  0.1128,  0.0779, -0.0790],
         [ 0.182

In [6]:
# %%
train_ds, val_ds = random_split(dataset, [0.8, 0.2])


# %%
len(train_ds)

# %%
len(val_ds)

# %%
train_loader = DataLoader(train_ds, batch_size = 16, shuffle=True)
test_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

In [7]:
from torch import nn

resnet50_model = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.DEFAULT)

resnet50_model.fc = nn.Identity()

# %%
for param in resnet50_model.parameters():
    param.requires_grad = False

resnet50_model.eval()

# %%
fc_model = nn.Sequential(
    nn.Linear(2048, 1024), 
    nn.ReLU(),
    nn.Linear(1024, 1)
)

# %%
model = nn.Sequential(
    resnet50_model, 
    fc_model
)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 210MB/s] 


In [8]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move model BEFORE creating optimizer
model = model.to(device)
resnet50_model = resnet50_model.to(device)

# # Define optimizer AFTER moving model
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

optimizer = torch.optim.Adam(fc_model.parameters(), lr = 0.001)

loss_fn = nn.BCEWithLogitsLoss()

Using device: cuda


In [9]:
num_epochs = 10
train_losses, val_losses = [], []
train_accs, val_accs = [], []

best_val_acc = 0.0
patience, patience_counter = 3, 0

In [10]:
for epoch in range(num_epochs):
    model.train()
    resnet50_model.eval()
    loss_sum = 0
    train_acc_sum = 0
    train_count = 0

    for x, y in train_loader:
        # Move data to GPU
        x = x.to(device)
        y = y.to(device)

        y = y.unsqueeze(1).float()
        outputs = model(x)
        optimizer.zero_grad()
        loss = loss_fn(outputs, y)
        loss_sum += loss.item()
        loss.backward()
        optimizer.step()
        
        prediction = torch.sigmoid(outputs) > 0.5
        acc = (prediction == y).sum().item()

        train_acc_sum += acc
        train_count += y.size(0)
        
    train_loss = loss_sum / len(train_loader)
    train_acc = train_acc_sum / train_count

    print(f"-----------------------Epoch {epoch + 1}---------------------------------")
    print("train loss:", loss_sum / len(train_loader))
    print("train_acc:", train_acc_sum / train_count)

    # Validation
    model.eval()
    test_loss_sum = 0
    test_acc_sum = 0
    test_count = 0
    
    with torch.no_grad():
        for x, y in test_loader:
            # Move data to GPU
            x = x.to(device)
            y = y.to(device)

            y = y.unsqueeze(1).float()
            outputs = model(x)
            loss = loss_fn(outputs, y)
            
            test_loss_sum += loss.item()
            predictions = torch.sigmoid(outputs) > 0.5
            acc = (predictions == y).sum().item()

            test_acc_sum += acc
            test_count += y.size(0)

    val_loss = test_loss_sum / len(test_loader)
    val_acc = test_acc_sum / test_count

        # ---- Save Metrics ----
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    print(f"----------------------Epoch : {epoch + 1} Validation-----------------")
    print("test loss:", val_loss)
    print("test acc total:", test_acc_sum)
    print("Test acc:", val_acc)


    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early Stopping triggered!")
            break 


-----------------------Epoch 1---------------------------------
train loss: 0.22021522975817162
train_acc: 0.898989898989899
----------------------Epoch : 1 Validation-----------------
test loss: 0.1767859299434349
test acc total: 340
Test acc: 0.9164420485175202
-----------------------Epoch 2---------------------------------
train loss: 0.08836072132051472
train_acc: 0.9656565656565657
----------------------Epoch : 2 Validation-----------------
test loss: 0.16021355839135745
test acc total: 347
Test acc: 0.9353099730458221
-----------------------Epoch 3---------------------------------
train loss: 0.03982127328485911
train_acc: 0.9858585858585859
----------------------Epoch : 3 Validation-----------------
test loss: 0.16511569162321393
test acc total: 345
Test acc: 0.9299191374663073
-----------------------Epoch 4---------------------------------
train loss: 0.013275570717846139
train_acc: 0.9973063973063973
----------------------Epoch : 4 Validation-----------------
test loss: 0.1624